In [ ]:
!pip install transformers datasets torch pandas

In [ ]:
!pip install transformers datasets torch pandas

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Interactly/Dataset/Resumes dataset/Resume/Resume.csv')

# Keep only relevant columns
df = df[['Resume_str', 'Category']]

# Encode labels
df['Category'] = df['Category'].astype('category')
df['label'] = df['Category'].cat.codes

# Create a Hugging Face dataset
dataset = Dataset.from_pandas(df[['Resume_str', 'label']])

# Split the dataset into train and test sets
dataset = dataset.train_test_split(test_size=0.2)

# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['Resume_str'], padding='max_length', truncation=True)

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Load a pre-trained model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['Category'].unique()))

# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Interactly/Fine Tuned LLM Model',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained('/content/drive/MyDrive/Interactly/Fine Tuned LLM Model/fine-tuned-model')
tokenizer.save_pretrained('/content/drive/MyDrive/Interactly/LLM Model/fine-tuned-model')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Map:   0%|          | 0/497 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.793051
2,No log,1.008190
3,2.015400,0.891855


('/content/drive/MyDrive/Interactly/LLM Model/fine-tuned-model/tokenizer_config.json',
 '/content/drive/MyDrive/Interactly/LLM Model/fine-tuned-model/special_tokens_map.json',
 '/content/drive/MyDrive/Interactly/LLM Model/fine-tuned-model/vocab.txt',
 '/content/drive/MyDrive/Interactly/LLM Model/fine-tuned-model/added_tokens.json',
 '/content/drive/MyDrive/Interactly/LLM Model/fine-tuned-model/tokenizer.json')